In [2]:
#eda import
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import sklearn

In [3]:
# csv file
benign = pd.read_csv('dataset/Benign Traffic.csv')
dos_udp_flood = pd.read_csv('dataset/DoS UDP Flood.csv')
dictionary_brute_force = pd.read_csv('dataset/Dictionary Brute Force.csv')
dos_dns_flood = pd.read_csv('dataset/DoS DNS Flood.csv')
dos_icmp_flood = pd.read_csv('dataset/DoS ICMP Flood.csv')
dos_syn_flood = pd.read_csv('dataset/DoS SYN Flood.csv')
mitm_arp_spoofing = pd.read_csv('dataset/MITM ARP Spoofing.csv')
recon_host_discovery = pd.read_csv('dataset/Recon Host Discovery.csv')
recon_os_scan = pd.read_csv('dataset/Recon OS Scan.csv')
recon_ping_sweep = pd.read_csv('dataset/Recon Ping Sweep.csv')
recon_port_scan = pd.read_csv('dataset/Recon Port Scan.csv')
recon_vul_scan = pd.read_csv('dataset/Recon Vulnerability Scan.csv')

In [7]:
csvs = [benign, dos_udp_flood, dictionary_brute_force, dos_dns_flood, dos_icmp_flood, dos_syn_flood, mitm_arp_spoofing, recon_host_discovery, recon_os_scan, recon_ping_sweep, recon_port_scan, recon_vul_scan]
# 각각 csv shape 확인
for csv in csvs:
    print(csv.shape)

(86525, 85)
(1848, 85)
(18151, 85)
(1702, 85)
(1405, 85)
(15243, 85)
(14768, 85)
(424, 85)
(42173, 85)
(47123, 85)
(453293, 85)
(39489, 85)
